# High-level Gluon Example

In [1]:
import os
import sys
import numpy as np
import math
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
from common.params import *
from common.utils import *

/anaconda/envs/py35/lib/python3.5/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
ctx = mx.gpu()

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("MXNet: ", mx.__version__)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
MXNet:  0.11.0
Numpy:  1.13.1
GPU:  ['Tesla M60', 'Tesla M60', 'Tesla M60', 'Tesla M60']


In [4]:
def SymbolModule():
    sym = gluon.nn.Sequential()
    with sym.name_scope():
        sym.add(gluon.nn.Conv2D(channels=50, kernel_size=3, padding=1, activation='relu'))
        sym.add(gluon.nn.Conv2D(channels=50, kernel_size=3, padding=1))
        sym.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        sym.add(gluon.nn.LeakyReLU(0))
        sym.add(gluon.nn.Dropout(0.25))
        sym.add(gluon.nn.Conv2D(channels=100, kernel_size=3, padding=1, activation='relu'))
        sym.add(gluon.nn.Conv2D(channels=100, kernel_size=3, padding=1))
        sym.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        sym.add(gluon.nn.LeakyReLU(0))
        sym.add(gluon.nn.Dropout(0.25))
        sym.add(gluon.nn.Flatten())
        sym.add(gluon.nn.Dense(512, activation='relu'))
        sym.add(gluon.nn.Dropout(0.25))
        sym.add(gluon.nn.Dense(N_CLASSES))
    return sym

In [5]:
def init_model(m):
    trainer = gluon.Trainer(m.collect_params(), 'sgd',
                            {'learning_rate': LR, 'momentum':MOMENTUM})
    criterion = gluon.loss.SoftmaxCrossEntropyLoss()
    return trainer, criterion

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Preparing train set...
Preparing test set...
Done.
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 2.93 s, sys: 1.44 s, total: 4.36 s
Wall time: 45.9 s


In [7]:
%%time
sym = SymbolModule()
sym.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

CPU times: user 596 ms, sys: 436 ms, total: 1.03 s
Wall time: 1.06 s


In [8]:
%%time
trainer, criterion = init_model(sym)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 349 µs


In [9]:
%%time
# Sets training = True 
for j in range(EPOCHS):
    train_loss = 0.0
    for data, target in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        # Get samples
        data = nd.array(data).as_in_context(ctx)
        target = nd.array(target).as_in_context(ctx)
        with autograd.record():
            # Forwards
            output = sym(data)
            # Loss
            loss = criterion(output, target)
        # Back-prop
        loss.backward()
        trainer.step(data.shape[0])
        train_loss += nd.sum(loss).asscalar()
    # Log
    print('Epoch %3d: loss: %5.4f'%(j, train_loss/len(x_train)))

Epoch   0: loss: 1.8455
Epoch   1: loss: 1.3743
Epoch   2: loss: 1.1377
Epoch   3: loss: 0.9633
Epoch   4: loss: 0.8310
Epoch   5: loss: 0.7190
Epoch   6: loss: 0.6322
Epoch   7: loss: 0.5528
Epoch   8: loss: 0.4768
Epoch   9: loss: 0.4092
CPU times: user 2min 2s, sys: 23.9 s, total: 2min 26s
Wall time: 1min 55s


In [10]:
%%time
# Test model
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, target in yield_mb(x_test, y_test, BATCHSIZE):
    # Get samples
    data = nd.array(data).as_in_context(ctx)
    # Forwards
    output = sym(data)
    pred = nd.argmax(output, axis=1)
    # Collect results
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = pred.asnumpy()
    c += 1

CPU times: user 820 ms, sys: 176 ms, total: 996 ms
Wall time: 775 ms


In [11]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.765324519231
